In [0]:
%sql
DROP SCHEMA IF EXISTS workspace.retail CASCADE;


In [0]:
%sql
SHOW SCHEMAS ;
    

databaseName
bronze
default
ecommerce
gold
information_schema
silver


In [0]:
!pip install kaggle


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 109.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()


In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "yerramvyshnavi"
os.environ["KAGGLE_KEY"] = "2bc801eeebec0558264a3c5e0d668d07"

print("✅ Kaggle credentials configured")


✅ Kaggle credentials configured


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.retail;

CREATE VOLUME IF NOT EXISTS workspace.retail.retail_data;


In [0]:
%sh
cd /Volumes/workspace/retail/retail_data

kaggle datasets download -d ulrikthygepedersen/online-retail-dataset


Dataset URL: https://www.kaggle.com/datasets/ulrikthygepedersen/online-retail-dataset
License(s): Attribution 4.0 International (CC BY 4.0)


100%|██████████| 7.38M/7.38M [00:01<00:00, 5.72MB/s]


In [0]:
%sh
cd /Volumes/workspace/retail/retail_data

unzip -o online-retail-dataset.zip
rm -f online-retail-dataset.zip
ls -lh


Archive:  online-retail-dataset.zip
  inflating: online_retail.csv       
total 47M
-rwxrwxrwx 1 spark-b38a4018-63c7-4e02-8dbc-a0 nogroup 47M Jan 28 17:28 online_retail.csv


In [0]:
df = spark.read.csv(
    "/Volumes/workspace/retail/retail_data/online_retail.csv",
    header=True,
    inferSchema=True
)

df.printSchema()


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
df.write.mode("overwrite").saveAsTable("workspace.retail.online_retail")


In [0]:
%sql
SELECT * 
FROM workspace.retail.online_retail
LIMIT 5;


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01T08:26:00.000Z,2.55,17850.0,United Kingdom
536365,71053,WHITE METAL LANTERN,6,2010-12-01T08:26:00.000Z,3.39,17850.0,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01T08:26:00.000Z,2.75,17850.0,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01T08:26:00.000Z,3.39,17850.0,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01T08:26:00.000Z,3.39,17850.0,United Kingdom


In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail.bronze_online_retail AS
SELECT * FROM workspace.retail.online_retail;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail.silver_online_retail AS
SELECT
  InvoiceNo,
  StockCode,
  Description,
  Quantity,
  InvoiceDate,
  DATE(InvoiceDate) AS order_date,
  UnitPrice,
  CustomerID,
  Country,
  Quantity * UnitPrice AS revenue
FROM retail.bronze_online_retail
WHERE Quantity > 0
  AND UnitPrice > 0
  AND CustomerID IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail.gold_daily_sales AS
SELECT
  order_date,
  ROUND(SUM(revenue),2) AS daily_revenue,
  SUM(Quantity) AS total_items_sold,
  COUNT(DISTINCT InvoiceNo) AS total_orders
FROM retail.silver_online_retail
GROUP BY order_date
ORDER BY order_date;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT order_date, daily_revenue
FROM retail.gold_daily_sales
ORDER BY order_date;


order_date,daily_revenue
2010-12-01,46376.49
2010-12-02,47316.53
2010-12-03,23921.71
2010-12-05,31771.6
2010-12-06,31215.64
2010-12-07,53795.31
2010-12-08,39248.82
2010-12-09,38231.9
2010-12-10,33650.28
2010-12-12,17305.77


In [0]:
%sql
SELECT order_date, total_orders, daily_revenue
FROM retail.gold_daily_sales;


order_date,total_orders,daily_revenue
2010-12-01,121,46376.49
2010-12-02,137,47316.53
2010-12-03,57,23921.71
2010-12-05,87,31771.6
2010-12-06,94,31215.64
2010-12-07,71,53795.31
2010-12-08,111,39248.82
2010-12-09,98,38231.9
2010-12-10,72,33650.28
2010-12-12,43,17305.77


In [0]:
%sql
SELECT order_date, daily_revenue
FROM retail.gold_daily_sales
ORDER BY daily_revenue DESC
LIMIT 10;


order_date,daily_revenue
2011-12-09,184349.28
2011-09-20,103435.81
2011-01-18,87589.11
2011-10-05,74108.43
2011-09-15,71993.67
2011-11-23,71979.93
2011-11-10,70513.29
2011-12-07,69354.21
2011-10-03,62275.95
2011-08-04,62079.61


In [0]:
df = spark.table("retail.gold_daily_sales").toPandas()

X = df[["total_items_sold", "total_orders"]]
y = df["daily_revenue"]


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [0]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [0]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("/Users/yerramvyshanavi@gmail.com/Sales_Forecasting")

with mlflow.start_run(run_name="linear_sales_forecast"):

    mlflow.log_param("model", "LinearRegression")
    mlflow.log_param("features", "total_items_sold,total_orders")
    mlflow.log_param("target", "daily_revenue")

    r2 = model.score(X_test, y_test)
    mlflow.log_metric("r2_score", r2)

    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        input_example=X_train.iloc[:5]
    )

    print(f"R2 Score: {r2:.3f}")


2026/02/01 16:20:13 INFO mlflow.tracking.fluent: Experiment with name '/Users/yerramvyshanavi@gmail.com/Sales_Forecasting' does not exist. Creating a new experiment.
/databricks/python/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

R2 Score: 0.919


In [0]:
df["predicted_revenue"] = model.predict(X)

pred_df = spark.createDataFrame(df)
pred_df.write.mode("overwrite").saveAsTable(
    "retail.gold_sales_forecast"
)


In [0]:
%sql
SELECT
  order_date,
  daily_revenue,
  predicted_revenue
FROM retail.gold_sales_forecast
ORDER BY order_date;


order_date,daily_revenue,predicted_revenue
2010-12-01,46376.49,42766.491228942934
2010-12-02,47316.53,54434.70663558162
2010-12-03,23921.71,20973.180128028664
2010-12-05,31771.6,29255.728924970615
2010-12-06,31215.64,29186.427468602767
2010-12-07,53795.31,33688.14620261348
2010-12-08,39248.82,38211.99458363165
2010-12-09,38231.9,31117.272943536766
2010-12-10,33650.28,28208.982497566452
2010-12-12,17305.77,18586.829961068524
